<a href="https://colab.research.google.com/github/Haebuk/dataminingTP/blob/main/TermProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 아파트 실거래가 예측

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from tqdm import tqdm
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import matplotlib as mpl  # 기본 설정 만지는 용도
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm
!apt-get update -qq
!apt-get install fonts-nanum* -qq
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)
fm._rebuild()
mpl.rcParams['axes.unicode_minus'] = False

NanumGothic Eco


## Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PATH = '/content/drive/MyDrive/input/dataminingTP/'
train_df = pd.read_csv(PATH + 'train.csv')
test_df = pd.read_csv(PATH + 'test.csv')

- exclusive_use_area: 전용면적(한 세대만 독점적으로 사용하는 공간)
- transaction_real_price: 실거래가(단위:만원, 타겟 값, train만 존재)
- transaction_year_month: 거래년월
- transaction_date: 거래일
- floor: 층 

- 데이터가 거래시간 순으로 이루어져 있음

## Preprocessing
### Train and Test set

In [ ]:
train_df.describe() 

,transaction_id,apartment_id,exclusive_use_area,year_of_completion,transaction_year_month,floor,transaction_real_price
count,1.216553e+06,1.216553e+06,1.216553e+06,1.216553e+06,1.216553e+06,1.216553e+06,1.216553e+06
mean,6.091530e+05,6.299685e+03,7.816549e+01,1.998297e+03,2.012984e+05,9.343291e+00,3.822769e+04
std,3.526198e+05,3.581169e+03,2.915113e+01,8.941347e+00,2.905732e+02,6.606500e+00,3.104898e+04
min,0.000000e+00,0.000000e+00,9.260000e+00,1.961000e+03,2.008010e+05,-4.000000e+00,1.000000e+02
25%,3.041380e+05,3.345000e+03,5.976000e+01,1.993000e+03,2.010100e+05,4.000000e+00,1.900000e+04
50%,6.082760e+05,5.964000e+03,8.241000e+01,1.999000e+03,2.013120e+05,8.000000e+00,3.090000e+04
75%,9.124140e+05,9.436000e+03,8.497000e+01,2.005000e+03,2.015110e+05,1.300000e+01,4.700000e+04
max,1.234827e+06,1.265800e+04,4.243200e+02,2.017000e+03,2.017110e+05,8.000000e+01,8.200000e+05


In [ ]:
train_df['apt'].value_counts()

현대          13154
한신           9766
삼성           6729
대우           6216
신동아          5851
            ...  
(750-6)         1
(740-62)        1
로미아트빌           1
금강빌리지           1
양지쉐르빌아파트        1
Name: apt, Length: 10440, dtype: int64

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216553 entries, 0 to 1216552
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   transaction_id          1216553 non-null  int64  
 1   apartment_id            1216553 non-null  int64  
 2   city                    1216553 non-null  object 
 3   dong                    1216553 non-null  object 
 4   jibun                   1216553 non-null  object 
 5   apt                     1216553 non-null  object 
 6   addr_kr                 1216553 non-null  object 
 7   exclusive_use_area      1216553 non-null  float64
 8   year_of_completion      1216553 non-null  int64  
 9   transaction_year_month  1216553 non-null  int64  
 10  transaction_date        1216553 non-null  object 
 11  floor                   1216553 non-null  int64  
 12  transaction_real_price  1216553 non-null  int64  
dtypes: float64(1), int64(6), object(6)
memory usage: 120.7+ M

- train_df 결측치는 없음
- 1961년 ~ 2017년 완공 건물
- 9.26m^2 ~ 424.32m^2 전용면적
- Floor의 경우 음의 값 존재.
- 실거래가는 100만원부터 820억까지 존재

In [ ]:
train_df.loc[train_df['transaction_real_price']==820000]

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
23603,23603,11320,서울특별시,한남동,810,한남더힐,한남동 810 한남더힐,244.749,2011,201612,21~31,3,820000


In [ ]:
# 같은 아파트 이름을 같는 수를 저장하는 apt_counts 열 생성
train_df['apt_counts'] = 0
train_df.groupby('apt')['apt_counts'].count()
train_df = pd.merge(train_df, train_df.groupby('apt')['apt_counts'].count(), on='apt', how='left').drop('apt_counts_x', axis=1).rename(columns={'apt_counts_y':'apt_counts'})

test_df['apt_counts'] = 0
test_df.groupby('apt')['apt_counts'].count()
test_df = pd.merge(test_df, test_df.groupby('apt')['apt_counts'].count(), on='apt', how='left').drop('apt_counts_x', axis=1).rename(columns={'apt_counts_y':'apt_counts'})

train_df.head(3)

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,apt_counts
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500,5
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000,1
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500,13


In [ ]:
train_df['top30'] = 0
top30 = ['자이', '푸르지오', '더샵', '롯데캐슬', '이편한|e편한|e-편한', '힐스테이트', '아이파크', '래미안', 'sk|SK|에스케이', '데시앙',
         '현대', '한신', '삼성', '대우', '신동아', '두신', '주공', '우성', '벽산', '동원로얄듀크',
         '경남', '삼환', '쌍용', '삼익', '대림', '코오롱', '파크리오', '엘지', '성원', '잠실']
for i, brand in enumerate(top30):
    train_df.loc[train_df['apt'].str.contains(brand), 'top30'] = 1
    train_df.loc[train_df['apt'].str.contains(brand), 'apt'] = str(i)
    test_df.loc[test_df['apt'].str.contains(brand), 'top30'] = 1
    test_df.loc[test_df['apt'].str.contains(brand), 'apt'] = str(i)

train_df.loc[(train_df['top30'] == 0), 'apt'] = 'others'
test_df.loc[(test_df['top30'] == 0), 'apt'] = 'others'

In [ ]:
all_df = pd.concat([train_df, test_df], axis=1).reset_index(drop=True)
all_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,apt_counts,top30,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,apt_counts,top30
0,0,7622,서울특별시,신교동,6-13,others,신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500,5,0,1145756.0,10453.0,서울특별시,목동,938,청학,목동 938 청학,35.5500,2002.0,201711.0,11~20,2.0,1.0,NaN
1,1,5399,서울특별시,필운동,142,others,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000,1,0,1198704.0,989.0,부산광역시,초량동,1143-8,고관맨션,초량동 1143-8 고관맨션,68.7200,1977.0,201708.0,21~31,2.0,1.0,NaN
2,2,3578,서울특별시,필운동,174-1,others,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500,13,0,1222384.0,8597.0,부산광역시,괴정동,447-13,우림그린,괴정동 447-13 우림그린,72.5400,1989.0,201710.0,11~20,2.0,1.0,NaN
3,3,10957,서울특별시,내수동,95,others,내수동 95 파크팰리스,146.39,2003,200801,11~20,15,118000,55,0,1179897.0,11086.0,서울특별시,대치동,1007-2,풍림아이원4차(1007-2),대치동 1007-2 풍림아이원4차(1007-2),111.5400,2004.0,201707.0,1~10,10.0,2.0,NaN
4,4,10639,서울특별시,내수동,110-15,others,내수동 110-15 킹스매너,194.43,2004,200801,21~31,3,120000,15,0,1223091.0,2121.0,부산광역시,다대동,1670,3,다대동 1670 다대롯데캐슬블루,119.6398,2014.0,201712.0,11~20,21.0,3.0,1.0


In [ ]:
train_df

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,apt_counts,top30
0,0,7622,서울특별시,신교동,6-13,others,신교동 6-13 신현(101동),84.8200,2002,200801,21~31,2,37500,5,0
1,1,5399,서울특별시,필운동,142,others,필운동 142 사직파크맨션,99.1700,1973,200801,1~10,6,20000,1,0
2,2,3578,서울특별시,필운동,174-1,others,필운동 174-1 두레엘리시안,84.7400,2007,200801,1~10,6,38500,13,0
3,3,10957,서울특별시,내수동,95,others,내수동 95 파크팰리스,146.3900,2003,200801,11~20,15,118000,55,0
4,4,10639,서울특별시,내수동,110-15,others,내수동 110-15 킹스매너,194.4300,2004,200801,21~31,3,120000,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216548,1234823,9578,부산광역시,정관읍 용수리,1387,others,정관읍 용수리 1387 정관 동일스위트1차,59.8245,2012,201711,11~20,3,22000,837,0
1216549,1234824,9578,부산광역시,정관읍 용수리,1387,others,정관읍 용수리 1387 정관 동일스위트1차,84.9923,2012,201711,21~30,7,30750,837,0
1216550,1234825,9578,부산광역시,정관읍 용수리,1387,others,정관읍 용수리 1387 정관 동일스위트1차,84.9923,2012,201711,21~30,9,29500,837,0
1216551,1234826,9586,부산광역시,정관읍 용수리,1364,3,정관읍 용수리 1364 정관신도시롯데캐슬,101.7400,2008,201711,21~30,7,26700,1020,1


In [ ]:
all_df[all_df['apt'].str.contains('송천')]

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,apt_counts,top30
1218968,1197684,6960,부산광역시,서대신동3가,544-2,송천,서대신동3가 544-2 송천,76.99,1976,201703,1~10,4,NaN,1,NaN


In [ ]:
# 완공연도에서 최소연도를 뺌으로써 완공연도 라벨인코딩
all_df['year_of_completion'] = all_df['year_of_completion'] - all_df['year_of_completion'].min()
all_df.head(1)

In [ ]:
# 거래연월에서 최소연월을 뺌으로써 거래연월 라벨 인코딩
all_df['transaction_year_month'] -= all_df['transaction_year_month'].min()
all_df.head(1)

In [ ]:
# 필요없는 열 제거
all_df = all_df.drop(['jibun', 'transaction_date', 'addr_kr', 'apt_counts', 'top30'], axis=1)
all_df.head(1)

#### Price

In [ ]:
# train price
plt.figure()
plt.hist(train_df['transaction_real_price'], bins=30)
plt.xlabel('Price(10000 won)')
plt.ylabel('Count')
plt.title('Distribution of Price')
plt.show()

- 가격의 분포가 매우 왼쪽으로 치우친 것을 확인할 수 있음
- 타겟 변수의 이상치가 회귀모형을 사용한 예측에 큰 영향을 줄 수 있음
    - 로그 변환으로 정규화한뒤, 나중에 다시 역변환

In [ ]:
# 가격 로그 변환 후 원래 가격 따로 저장
all_df['log_price'] = np.log1p(all_df['transaction_real_price'])
real_price = all_df['transaction_real_price']
train_y = real_price[:train_df.shape[0]]
test_y = real_price[train_df.shape[0]:]
all_df.drop('transaction_real_price', axis=1, inplace=True)
all_df.head(1)

In [ ]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum()
    print("Memory usage of dataframe in {:.2f} MB",format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                # np.iinfo: 정수형 타입의 데이터에 명시한 데이터 타입만큼의 메모리 할당
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
                # np.finfo
                elif c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float64).min and c_max < np.finfo(np.float64).max:
                    df[col] = df[col].astype(np.float64)
            else:
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum()
    print('Memory Usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

In [ ]:
ohe_col = ['city', 'dong', 'apt']
onehot_columns = pd.get_dummies(all_df[ohe_col])
all_df = pd.concat([all_df, onehot_columns], axis=1)

In [ ]:
apt_id = all_df['apartment_id']
all_df.drop(['transaction_id', 'apartment_id', 'city', 'dong'], axis=1, inplace=True)
all_df.head(1)

In [ ]:
train_df = all_df[:train_df.shape[0]]
test_df = all_df[train_df.shape[0]:]
del all_df

## Modeling

In [ ]:
NFOLDS = 5

def rmsle_cv(model):
    rmse = np.sqrt(-cross_val_score(model, train_df.values, train_y, scoring='neg_mean_squared_error'))
    return(rmse)

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.0005, random_state=1))
score = rmsle_cv(lasso)
print('\nLasso score: {:.4f} ({:.4f})\n'.format(score.mean(), score.std()))

#### Area

In [ ]:
# train area
plt.figure()
plt.hist(train_df['exclusive_use_area'], bins=30)
plt.xlabel('Area(Square meter)')
plt.ylabel('Count')
plt.title('Distribution of Area')
plt.show()

- 면적도 마찬가지로 정규화

#### What is Underground Floor ?

In [ ]:
under_floor = train_df.loc[train_df['floor']<0]
print(f'총 {under_floor.shape[0]}개의 음수 층 있음')
print(under_floor['floor'].value_counts())

- 168개의 지하층이 존재함

In [ ]:
plt.figure()
sns.countplot(under_floor['floor'])
plt.title('')
plt.show()

In [ ]:
under_floor[['floor', 'transaction_real_price']].corr()

층수와 가격간의 선형 설명력이 어느정도 존재하기 때문에 제거하지 않는다.

In [ ]:
test_df.head()

In [ ]:
test_df.describe()

In [ ]:
test_df.info()

- test셋에도 널값은 존재하지 않음.
- 실거래가 변수 없음(타겟 변수)
- 완공연도 1961년부터 2017년
- 실거래연도는 2017년도만 존재함
- 전용면적은 10.32m^2부터 273.86m^2까지 존재

In [ ]:
plt.figure(figsize=(8, 8))
sns.countplot(test_df['transaction_year_month'])
plt.xticks(rotation=60)
plt.show()

2017년 데이터만 존재하며, 12월 데이터가 가장 많음

- 시설에 매우 많은 결측치가 존재함
- 시설이 존재하면 1 없으면 0으로 값 대체


- 분포가 매우 왼쪽으로 치우친 형태
- 공원 면적이 집 값에 영향을 미치는가?